<a href="https://colab.research.google.com/github/bharatbajoria/Summer-Internship/blob/master/KL_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow 
import pandas as pd
import numpy as npd

In [ ]:
pip install bert-extractive-summarizer

In [ ]:
pip install spacy==2.1.3

In [ ]:
pip install transformers==2.2.2

In [ ]:
pip install neuralcoref

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2

In [ ]:
def lcs(X,Y,m,n,Corp_Participants):
  #X,Y=removing_corp(X,Y,Corp_Participants)
  x="BERT-Presentation"
  y="KL-Presentation"
  if x in (X or Y):
    X=X.replace(x,"")
    Y=Y.replace(x,"")
  if y in (Y or X):
    Y=Y.replace(y,"")
    X=X.replace(x,"")
  if X[0]==" ":
    X=X[1:]
  if Y[0]==" ":
    Y=Y[1:]

  X=data_clean(X)
  Y=data_clean(Y)
  X=X.split(".")
  Y=Y.split(".")
  common_text=""
  for i in X:
    if (i in Y and len(word_tokenize(i))>3):
      common_text+=i+"."
  
  return common_text

def finding_complement(X,Y,Corp_Participants):
  #returns Y-X
  X,Y=removing_corp(X,Y,Corp_Participants)
  x="BERT-Presentation"
  y="KL-Presentation"
  if x in (X or Y):
    X=X.replace(x,"")
    Y=Y.replace(x,"")
  if y in (Y or X):
    Y=Y.replace(y,"")
    X=X.replace(x,"")
  if (len(X)>0 and len(Y)>0):
    if X[0]==" ":
      X=X[1:]
    if Y[0]==" ":
      Y=Y[1:]  
    X=data_clean(X)
    Y=data_clean(Y)
    X=X.split(".")
    Y=Y.split(".")
    common_text=[]
    for i in X:
      if i in Y:
        common_text.append(i)
    
    for i in common_text:
      if i in Y :
        index_i=Y.index(i)
        Y.pop(index_i)
    
    text=""
    for i in Y:
      text+=i+"."

    return text
  else:
    return data_clean(Y)


In [ ]:
def data_clean(train_data):
  train_data=train_data.replace("\n\n"," ")
  train_data=train_data.replace("\n"," ")
  train_data=train_data.replace(", "," ")
  train_data=train_data.replace(" ,"," ")
  train_data=train_data.replace(","," ")
  train_data=train_data.replace("-"," ")
  train_data=train_data.replace("    "," ")
  train_data=train_data.replace("   "," ")
  train_data=train_data.replace("  "," ") 
  train_data=train_data.replace("<Sentence:","")
  train_data=train_data.replace(">","")
  train_data=train_data.replace("(","")
  train_data=train_data.replace(")","")
  train_data=train_data.replace(".,",".")

  return train_data



In [ ]:
# creating a pdf file object 
#files=["Ali Baba.pdf","Alphabet.pdf","Amazon.pdf","Apple.pdf","Baidu.pdf","FB.pdf","Netflix.pdf"]
files=["Amazon.pdf","Apple.pdf"]
pagewise_data=[]#2-D data
All_docs=[]#All_docs[0] is a list of string of first file
#pagewise_data[0] list of page wise string of first file-Ali Baba
All_docs_corporate_participants=[]# a list of strings of corporate participants

All_docs_questions=[]#2-D list
#All_docs_questions[0] is a list of all questions of first file 
All_docs_answers=[]
All_docs_presentations=[]
for file_count in range(len(files)):

  pdfFileObj = open(files[file_count],'rb') 
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  pages=pdfReader.numPages
  data=[]
  for i in range(pages): 
    pageObj = pdfReader.getPage(i) 
    data.append(pageObj.extractText())
  pagewise_data.append(data)
  All_data=[]
  data_str=''
  for i in range(len(data)):
    data_str+=data[i]
  
  data_str=data_str.replace("\n","")
  
  All_data=[data_str]
  All_docs.append(All_data)
  questions=[]
  answers=[]
  
  corporate_Participants_str="CORPORATE PARTICIPANTS "
  presentation_str="PRESENTATION "
  QnA_str="QUESTIONS AND ANSWERS"

  index_i=data_str.index(corporate_Participants_str)
  index_j=data_str.index(presentation_str)
  
  All_docs_corporate_participants.append(data_str[index_i:index_j])
  
  index_i=data_str.index(QnA_str)
  All_docs_presentations.append(data_str[index_j:index_i])
  data_str=data_str[index_i:] #Retaining only QnA  



  count_qna=min(data_str.count("Question:"),data_str.count("Answer:"))
  for count in range(count_qna):   
    i=data_str.index("Question:")
    j=data_str.index("Answer:")
    dup_data_str=data_str[i:j]
    questions.append(dup_data_str)   
    data_str=data_str[j:]
    
    i=data_str.index("Answer:")
    if count!= count_qna-1:
      j=data_str.index("Question:")
      dup_data_str=data_str[i:j]
      x="Answer-"+str(count+1)+" :"
      dup_data_str=dup_data_str.replace("Answer:",x)
    else:
      dup_data_str=data_str[i:]
      x="Answer-"+str(count+1)+" :"
      dup_data_str=dup_data_str.replace("Answer:",x)
    
    answers.append(dup_data_str)#appending answers
    data_str=data_str[j:]
  
  
  All_docs_questions.append(questions)
  All_docs_answers.append(answers) 

In [ ]:


def preparing_presentation_data(doc,corp_participant):
  corp_participant= corp_participant.split("  ")
  for i in range(len(corp_participant)):
    if corp_participant[i][-1]==" ":    
      corp_participant[i]=corp_participant[i][0:(len(corp_participant[i])-1)]

  corp_participant=corp_participant[1:]

  participant_index=[]
    
  for i in range(len(corp_participant)):
    if doc.count(corp_participant[i])>0:
      j= doc.index(corp_participant[i])
      participant_index.append(j)
    else:
      participant_index.append(0)

  dup_participant_index=participant_index[:]
  dup_corp_participant=[]
  dup_participant_index.sort()

  for i in dup_participant_index:
    if i!=0:
      index_i= participant_index.index(i)
      dup_corp_participant.append(corp_participant[index_i])

  presentation_doc=[]

  for i in range(len(dup_corp_participant)-1):
    index_i=doc.index(dup_corp_participant[i])
    index_j=doc.index(dup_corp_participant[i+1])
    x=doc[index_i:index_j]
    x=x.replace(dup_corp_participant[i],dup_corp_participant[i]+"\n\n")
    presentation_doc.append(x)
    if i+1==len(dup_corp_participant)-1:
      x=doc[index_j:]
      x=x.replace(dup_corp_participant[i+1],dup_corp_participant[i+1]+"\n\n")
      presentation_doc.append(x)
    


  if len(dup_corp_participant)==1:
    
    index_i=doc.index(dup_corp_participant[0])
    x=doc[index_i:]
    x=x.replace(dup_corp_participant[0],dup_corp_participant[0]+"\n\n")
    presentation_doc.append(x)

    
  return presentation_doc,dup_corp_participant


In [ ]:
presentation_doc,corp_participant=preparing_presentation_data(All_docs_presentations[1],All_docs_corporate_participants[1])

In [ ]:
x=All_docs_corporate_participants[0]
x=x.split("  ")
x=x[1:]
Corp_Participants=x
Corp_Participants

['Brian T. Olsavsky Amazon.com, Inc. - Senior VP & CFO',
 'Dave Fildes Amazon.com, Inc. - Director of IR ']

#KL Summary

In [ ]:
pip install sumy

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 10.1MB 8.7MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=8d3fd51af2c8845f53dd3a64e1af84180cac33ad1df87e8a67c729f0fec186ec
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21684 sha256=823da5202ea245e2bd7dd60b603672d1551e5fdf49ee9d913613529c4782ea9f
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability


In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.kl import KLSummarizer
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
All_docs_summarized=[]
All_docs_KL_summarized=[]
Percent_breakup=0.60

def LuhnSumm(doc,number_sent,Percent_breakup):
  parser=PlaintextParser.from_string(doc,Tokenizer("english"))
  summary=''
  summarizer = KLSummarizer()
  #Summarize the document with 4 sentences
  k= int(Percent_breakup*number_sent)
  summary = summarizer(parser.document,k)
  
  return summary

for i in range(len(files)):
  doc_summary=[]
  answer_summary=[]
  presentation_summary=[]  
  pres,partp=All_docs_presentations[i],All_docs_corporate_participants[i]
  presentation_doc,corp_participant=preparing_presentation_data(pres,partp)
  j=0
  for speaker_data in presentation_doc:
    numb_sent=len(sent_tokenize(speaker_data))
    presentation_summ=LuhnSumm(speaker_data,numb_sent,Percent_breakup)
    presentation_summ=str(presentation_summ)
    if presentation_summary.count(corp_participant[j])==0:
      presentation_summ= corp_participant[j]+" " + presentation_summ[:]
    j+=1
    presentation_summ=presentation_summ+" \n"
    presentation_summary.append(presentation_summ)

  for j in range(len(All_docs_answers[i])):
    numb_sent=len(sent_tokenize(All_docs_answers[i][j]))    
    content_sent=sent_tokenize(All_docs_answers[i][j])    
    summary = LuhnSumm(All_docs_answers[i][j],numb_sent,Percent_breakup)
    summary=str(summary)
    if summary.count(content_sent[0])==0:
      summary=content_sent[0]+summary
    x="Answer"+"-"+str(j+1)+" :"
    summary=summary.replace("Answer:",x)
    y=summary
    y=y.replace("<Sentence:","")
    y=y.replace(">","")
    y=y.replace("(","")
    y=y.replace(")","")
    y=y.replace(".,",".")

    answer_summary.append(y)
  
  doc_summary.append(files[i])
  doc_summary.append(presentation_summary)

  for j in range(len(All_docs_answers[i])):
    x= "Question"+"-"+str(j+1)+" :"
    All_docs_questions[i][j]=All_docs_questions[i][j].replace("Question:",x)
    doc_summary.append(All_docs_questions[i][j])
    doc_summary.append(answer_summary[j])
  
  All_docs_KL_summarized.append(doc_summary)

#Fuzzy String Matching

In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
Summarized_ans=All_docs_KL_summarized[1][2:]
Summarized_ans=Summarized_ans[1::2]

upper_index=[]
fuzz_cutoff=90
for i in range(len(Summarized_ans)-1):
  lower_index=[]
  for j in range(i,len(Summarized_ans)):
    fuzz_score=fuzz.ratio(Summarized_ans[i],Summarized_ans[j])
    if fuzz_score>=fuzz_cutoff:
      if len(Summarized_ans[i])>len(Summarized_ans[j]):
        lower_index.append(j)
      elif i!=j:
        upper_index.append(i)
  
  lower_index.sort(reverse=True)  
  for j in range(len(lower_index)):
    Summarized_ans.pop(lower_index[j])

  

In [ ]:

Summarized_ans=All_docs_KL_summarized[0][2:]
Summarized_ans=Summarized_ans[1::2]
Ans_index=[]
x="Ans-"
Fuzz_score=[]
fuzz_cutoff=0.9
for i in range(len(Summarized_ans)):
  Var_fuzz_scr=[]
  for j in range(0,len(Summarized_ans)):
    fuzz_score=fuzz.ratio(Summarized_ans[i],Summarized_ans[j])
    Var_fuzz_scr.append(fuzz_score)
  Fuzz_score.append(Var_fuzz_scr)
  y=x+str(i+1)
  Ans_index.append(y)

Fuzz_df=pd.DataFrame(data=Fuzz_score, columns=Ans_index,index=Ans_index)
#Fuzz_df



In [ ]:

def preparing_data_PDFwrite(docs):
  x='Presentaion\n\n'
  count_for_qna=0
  
  for i in range(1,len(docs)):
    if i==1:
      for k in docs[i]:
        y=str(k)
        if y.count("<Sentence:")>0:
          y=y.replace("<Sentence:","")
          y=y.replace(">","")
          y=y.replace("(","")
          y=y.replace(")","")
          y=y.replace(".,",".")
        x+=y +"\n"
      x+="\nQnA\n\n"
    else:
      y=str(docs[i])
      if y.count("<Sentence:")>0:
        y=y.replace("<Sentence:","")
        y=y.replace(">","")
        y=y.replace("(","")
        y=y.replace(")","")
        y=y.replace(".,","")
           
      x+=y
      x+="  \n\n"
      
      
  return x


In [ ]:
pip install python-docx

#BERT Summ

In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
#https://pypi.org/project/bert-extractive-summarizer/

All_docs_answers[0]

["Answer-1 : Dave Fildes Amazon.com, Inc. - Director of IR Justin, this is Dave. On the second question, on the subscription service, we'd said on the last call, you probably remember, related to the adoption of the Accounting Standards Update and revenue recognition policies that was impacting our results in a number of areas in 2018, we'd said we'd anticipated about $300 million lower sort of headwind to subscription services revenue due to the accounting change. So that's where we came out for the quarter. So you'll see that we reported subscription services revenue increased 26%, when you exclude FX. So the $300 million is certainly part of that decelerate, a sequential deceleration in the growth rate in that line item. Brian T. Olsavsky Amazon.com, Inc. - Senior VP & CFO Yes, and on unit deceleration, I think the unit numbers more and more require some interpretation because it doesn't include the fast -- some of our fastest growing areas, things like, as Dave mentioned, subscript

In [ ]:
All_docs_presentations

["PRESENTATION  Operator Thank you for standing by. Good day, everyone, and welcome to the Amazon.com Q4 2018 Financial Results Teleconference. (Operator Instructions) Today's call is being recorded. For opening remarks, I'll be turning the call over to the Director of Investor Relations, Dave Fildes. Please go ahead. Dave Fildes Amazon.com, Inc. - Director of IR Hello, and welcome to our Q4 2018 financial results conference call. Joining us today to answer your questions is Brian Olsavsky, our CFO. As you listen to today's conference call, we encourage you to have our press release in front of you, which includes our financial results as well as metrics and commentary on the quarter. Please note, unless otherwise stated, all comparisons in this call will be against our results for the comparable period of 2017. Our comments and responses to your questions reflect management's views as of today, January 31, 2019, only and will include forward-looking statements. Actual results may diff

In [ ]:
All_docs_questions[0]

["Question-1 : Justin Post BofA Merrill Lynch, Research Division - MD I guess I'll ask about units, 14% in the quarter. Just how do you feel about the overall unit growth here given that growth was higher last year in the 20s? And do you think there's some investments you can make in other areas to kind of reaccelerate that going forward? And then, secondly, just if you could remind us or help us understand the Prime accounting change impact on subscription revenues in Q4? ",
 "Question-2 : Mark Stephen F. Mahaney RBC Capital Markets, LLC, Research Division - MD and Analyst Do you want to just comment a little bit on the international revenue outlook and any commentary on India and whether you think there's a material impact on your business? Secondly, could you talk about advertising revenue, just qualitatively or quantitatively, how that's doing? And third, if I could, you spent a lot on marketing in the quarter. It's a real step up. Do you want to just talk about the ROI that you th

In [ ]:
def Bert_summ(docs,Percentage_breakup):
  result=model(docs,ratio=Percentage_breakup)
  return "".join(result)


In [ ]:
All_docs_Bert_summarized=[]

for i in range(len(files)):
  doc_summary=[]
  answer_summary=[]
  presentation_summary=[]  
  pres,partp=All_docs_presentations[i],All_docs_corporate_participants[i]
  presentation_doc,corp_participant=preparing_presentation_data(pres,partp)
  j=0
  for speaker_data in presentation_doc:
    
    presentation_summ=Bert_summ(speaker_data,Percent_breakup)
    presentation_summ=str(presentation_summ)
    if presentation_summary.count(corp_participant[j])==0:
      presentation_summ= "\n"+corp_participant[j] + presentation_summ[:]
    j+=1
    
    presentation_summary.append(presentation_summ)

  for j in range(len(All_docs_answers[i])):
    content_sent=sent_tokenize(All_docs_answers[i][j])    
    summary = Bert_summ(All_docs_answers[i][j],Percent_breakup)
    summary=str(summary)
    if summary.count(content_sent[0])==0:
      summary=content_sent[0]+summary
    x="Answer"+"-"+str(j+1)+" :"
    summary=summary.replace("Answer:",x)
    y=summary
    y=y.replace("<Sentence:","")
    y=y.replace(">","")
    y=y.replace("(","")
    y=y.replace(")","")
    y=y.replace(".,",".")

    answer_summary.append(y)

  doc_summary.append(files[i])
  doc_summary.append(presentation_summary)

  for j in range(len(All_docs_answers[i])):
    doc_summary.append(All_docs_questions[i][j])
    doc_summary.append(answer_summary[j])
  
  All_docs_Bert_summarized.append(doc_summary)

In [ ]:
def preparing_final_data(Bert_data,KL_data,doc_pres,Doc_Answers):
  #Bert_data is str of summarised BERT Data
  #KL_data is str of summarised KL Data
  Corp="CORPORATE PARTICIPANTS"
  Pres="Presentaion"
  QnA="QnA"
  data=""
  index_corp=Bert_data.index(Corp)
  index_pres= Bert_data.index(Pres)
  data+="\n"+Bert_data[index_corp:index_pres]#Adding names of Corp. Participant
  
  #Adding Presentation Bert then KL
  index_pres= Bert_data.index(Pres)
  index_QnA= Bert_data.index(QnA)
  Bert_Pres=Bert_data[index_pres:index_QnA]
  Bert_Pres=Bert_Pres.replace("Presentaion\n","BERT-Presentation")
  data+="PRESENTATION\n\n"+"Original Presentation-\n"+doc_pres+"\n\n"+Bert_Pres

  index_pres= KL_data.index(Pres)
  index_QnA= KL_data.index(QnA)
  KL_Pres=KL_data[index_pres:index_QnA]
  KL_Pres=KL_Pres.replace("Presentaion\n","KL-Presentation")
  data+="\n"+KL_Pres

  #Adding QnA Header
  QnA_count=Bert_data.count("Question-")
  data+="\n\nQnA\n\n"
  for i in range(QnA_count):
    question_id= "Question-"+str(i+1)
    ans_id= "Answer-"+str(i+1)
    #Adding question
    index_ques= Bert_data.index(question_id)
    index_ans= Bert_data.index(ans_id)
    data+="\n\n"+Bert_data[index_ques:index_ans]

    #Adding Bert Answer then KL Answer
    if i!= QnA_count-1:
      question_id= "Question-"+str(i+2)
      ans_id= "Answer-"+str(i+1)    
      index_ques= Bert_data.index(question_id)
      index_ans= Bert_data.index(ans_id)
      data+="\nOriginal Answer-\n"+Doc_Answers[i] +"\nBERT:\n"+Bert_data[index_ans:index_ques]
    else:
      
      ans_id= "Answer-"+str(i+1)    
      
      index_ans= Bert_data.index(ans_id)
      data+="\nOriginal Answer-\n"+Doc_Answers[i]+ "\n\nBERT:\n"+Bert_data[index_ans:]
#Adding KL Answer
    if i!= QnA_count-1:
      question_id= "Question-"+str(i+2)
      ans_id= "Answer-"+str(i+1)    
      index_ques= KL_data.index(question_id)
      index_ans= KL_data.index(ans_id)
      data+="\nKL:\n"+KL_data[index_ans:index_ques]
    else:
      
      ans_id= "Answer-"+str(i+1)    
      
      index_ans= KL_data.index(ans_id)
      data+="\nKL:\n"+KL_data[index_ans:]
  
  return data

In [ ]:
def removing_corp(Bert_data,KL_data,Corp_Participants):
  if "BERT-Presentation" in Bert_data:
    Bert_data=Bert_data.replace("BERT-Presentation ","")
  if "KL-Presentation" in KL_data:
    KL_data=KL_data.replace("KL-Presentation ","")
  
  for i in Corp_Participants:
    if i in Bert_data:
      Bert_data=Bert_data.replace(i," ")
    if i in KL_data:
      KL_data=KL_data.replace(i," ")
  
  return Bert_data,KL_data

def preparing_final_data_test(Bert_data,KL_data,doc_pres,Doc_Answers,Corp_Participants):
  #Bert_data is str of summarised BERT Data
  #KL_data is str of summarised KL Data
  
  Corp="CORPORATE PARTICIPANTS"
  Pres="Presentaion"
  QnA="QnA"
  data=""
  index_corp=Bert_data.index(Corp)
  index_pres= Bert_data.index(Pres)
  #Corp_Participants=Bert_data[index_corp:index_pres][:]
  data+="\n"+Bert_data[index_corp:index_pres]#Adding names of Corp. Participant

  Corp_Participants=Corp_Participants.split("  ")
  Corp_Participants=Corp_Participants[1:]
  x=Corp_Participants[-1]
  
  Corp_Participants[-1]= x[0:(len(Corp_Participants[-1])-1)]

  #Adding Presentation Bert then KL
  index_pres= Bert_data.index(Pres)
  index_QnA= Bert_data.index(QnA)
  Bert_Pres=Bert_data[index_pres:index_QnA]
  Bert_Pres=Bert_Pres.replace("Presentaion\n","BERT-Presentation")
  data+="PRESENTATION\n\n"+"Original Presentation-\n"+"Sentence Count -"+str(len(doc_pres.split("."))) +"\n"
  data+=doc_pres+"\n\n"+"Sentence Count -"+str(len(Bert_Pres.split("."))) +"\n"+Bert_Pres

  index_pres= KL_data.index(Pres)
  index_QnA= KL_data.index(QnA)
  KL_Pres=KL_data[index_pres:index_QnA]
  KL_Pres=KL_Pres.replace("Presentaion\n","KL-Presentation")  
  data+="\n"+"Sentence Count -"+str(len(KL_Pres.split("."))) +"\n"+KL_Pres
# Adding 4 sets of KL and Bert and complements
  Bert_Pres,KL_Pres=removing_corp(Bert_Pres,KL_Pres,Corp_Participants)
  #removing corporate names
  Pres_bert_and_kl=lcs(Bert_Pres,KL_Pres,len(Bert_Pres),len(KL_Pres),Corp_Participants)  
  Pres_bert_and_klc=finding_complement(Pres_bert_and_kl,Bert_Pres,Corp_Participants)
  
  Pres_bertc_and_kl=finding_complement(Pres_bert_and_kl,KL_Pres,Corp_Participants)
  Pres_bertc_and_kl=finding_complement(Pres_bert_and_klc,Pres_bertc_and_kl,Corp_Participants)
  
  Pres_bertc_and_klc= finding_complement(Bert_Pres,doc_pres,Corp_Participants)
  Pres_bertc_and_klc= finding_complement(Pres_bertc_and_kl,Pres_bertc_and_klc,Corp_Participants)
  Pres_bertc_and_klc= finding_complement(Pres_bert_and_kl,Pres_bertc_and_klc,Corp_Participants)
  data+="\n" +"KL and Bert\n\n"   + Pres_bert_and_kl
  data+="\n\n"+"Bert~ and KL\n\n" + Pres_bertc_and_kl
  data+="\n\n"+"Bert and KL~\n\n" + Pres_bert_and_klc
  data+="\n\n"+"Bert~ and KL~\n\n"+ Pres_bertc_and_klc



  #Adding QnA Header
  QnA_count=Bert_data.count("Question-")
  data+="\n\nQnA\n\n"
  for i in range(QnA_count):
    question_id= "Question-"+str(i+1)
    ans_id= "Answer-"+str(i+1)
    #Adding question
    index_ques= Bert_data.index(question_id)
    index_ans= Bert_data.index(ans_id)
    data+="\n\n"+Bert_data[index_ques:index_ans]

    #Adding Bert Answer then KL Answer
    if i!= QnA_count-1:
      question_id= "Question-"+str(i+2)
      ans_id= "Answer-"+str(i+1)    
      index_ques= Bert_data.index(question_id)
      index_ans= Bert_data.index(ans_id)
      Bert_ans=Bert_data[index_ans:index_ques]
      sent_count_doc=len(Doc_Answers[i].split("."))
      sent_count_bert=len(Bert_ans.split("."))
      data+="\nOriginal Answer-\n"+"Sentence Count-"+str(sent_count_doc)+"\n"+Doc_Answers[i]
      data+= "\n"+"Sentence Count-"+str(sent_count_bert)+"\nBERT:\n"+Bert_ans
    
    else:     
      ans_id= "Answer-"+str(i+1)     
      index_ans= Bert_data.index(ans_id)
      Bert_ans=Bert_data[index_ans:]
      sent_count_doc=len(Doc_Answers[i].split("."))
      sent_count_bert=len(Bert_ans.split("."))
      data+="\nOriginal Answer-\n"+"Sentence Count-"+str(sent_count_doc)+"\n"+Doc_Answers[i]
      data+="\n"+"Sentence Count-"+str(sent_count_bert)+"\nBERT:\n"+Bert_ans
#Adding KL Answer
    if i!= QnA_count-1:
      question_id= "Question-"+str(i+2)
      ans_id= "Answer-"+str(i+1)    
      index_ques= KL_data.index(question_id)
      index_ans= KL_data.index(ans_id)
      KL_ans=KL_data[index_ans:index_ques]
      sent_count_kl=len(KL_ans.split("."))
      data+="\nKL:\n"+"Sentence Count"+str(sent_count_kl)+"\n"+KL_ans
    else:      
      ans_id= "Answer-"+str(i+1)         
      index_ans= KL_data.index(ans_id)
      KL_ans=KL_data[index_ans:]
      sent_count_kl=len(KL_ans.split("."))
      data+="\nKL:\n"+"Sentence Count"+str(sent_count_kl)+"\n"+KL_ans
    
    Bert_ans,KL_ans=removing_corp(Bert_ans,KL_ans,Corp_Participants)
    Ans_bert_and_kl=lcs(Bert_ans,KL_ans,len(Bert_ans),len(KL_ans),Corp_Participants)  
    Ans_bert_and_klc =finding_complement(Ans_bert_and_kl,Bert_ans,Corp_Participants)
    
    Ans_bertc_and_kl =finding_complement(Ans_bert_and_kl,KL_ans,Corp_Participants)
    Ans_bertc_and_kl =finding_complement(Ans_bert_and_klc,Ans_bertc_and_kl,Corp_Participants)    
    
    Ans_bertc_and_klc= finding_complement(Bert_ans, Doc_Answers[i],Corp_Participants)
    Ans_bertc_and_klc= finding_complement(Ans_bertc_and_kl,Ans_bertc_and_klc,Corp_Participants)
    Ans_bertc_and_klc= finding_complement(Ans_bert_and_kl,Ans_bertc_and_klc,Corp_Participants)
    data+="\n" +"KL and Bert\n\n"   + Ans_bert_and_kl
    data+="\n\n"+"Bert~ and KL\n\n" + Ans_bertc_and_kl
    data+="\n\n"+"Bert and KL~\n\n" + Ans_bert_and_klc
    data+="\n\n"+"Bert~ and KL~\n\n"+ Ans_bertc_and_klc


  
  return data

In [ ]:
#Generating files
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH

for i in range(len(files)):
  print("File Number:",i)
  Heading="Summary "+files[i]
  Heading=Heading.replace(".pdf","")
  
  y=All_docs_corporate_participants[i]
  y=y.replace("CORPORATE PARTICIPANTS","\nCORPORATE PARTICIPANTS\n")
  y=y.replace("  ","\n")

  #Preparing KL Data
  output_data_kl=y+"\n\n"+preparing_data_PDFwrite(All_docs_KL_summarized[i])
  
  #Preparing Bert Data  
  output_data_bert=y+"\n\n"+preparing_data_PDFwrite(All_docs_Bert_summarized[i])
  doc = docx.Document()    
  # add a heading of level 0 (largest heading) 
  doc.add_heading(Heading, 1)
  
  output_data= preparing_final_data_test(output_data_bert,
                                         output_data_kl,
                                         All_docs_presentations[i],
                                         All_docs_answers[i],
                                         All_docs_corporate_participants[i])
  
  doc_para = doc.add_paragraph(output_data) 

  #doc_para.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

  doc.save(Heading)



File Number: 0
File Number: 1


#Misc. moved to Untitled-19